In [5]:
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.lines import Line2D
from matplotlib import rc
import seaborn as sns
# plt.rcParams['ps.useafm'] = True
# rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})

import pandas as pd
import time
import shutil
import argparse
import numpy as np
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import StandardScaler
import sys
# sys.path.append('/DATA/lucaa/software/CLoNe')

# from clone import CLoNe
# from plot import plot_clusters
# from structural_utils import load_md_args, show_cluster_info

import mdtraj
import pyemma

print(__doc__)
start = time.time()


def calculate_pca(topo, traj, main_folder, syst, at_sel, n_pca ):
    # Load trajectory
    topology = mdtraj.load(topo).topology
    struct_ens = mdtraj.load(traj, top=topo)

    # Create result folder based on dataset nam\e
    if not os.path.exists(main_folder):
        os.makedirs(main_folder)
    out_idx =  1
    output_folder = main_folder+"/%s_%i/"
    while os.path.exists(output_folder%(syst, out_idx)):
        out_idx += 1
    output_folder = output_folder%(syst, out_idx)
    os.makedirs(output_folder)
    

    selection = struct_ens.topology.select(at_sel)
    coords = struct_ens.xyz[:, selection]
    coords = coords.reshape(coords.shape[0], coords.shape[1] * coords.shape[2])
    headers = ["C%i"%x for x in range(len(coords[0]))]  # general headers

    # Principal component analysis
    original_coords = coords.copy()
    pca_obj = PCA(n_components=n_pca)
    reddim_coords = pca_obj.fit_transform(coords)
    eigenvalues = pca_obj.explained_variance_ratio_
    ratio = np.sum(eigenvalues[:n_pca])
    pca_headers = ["PC%i (%.2f)"%(x + 1, eigenvalues[x]) for x in range(n_pca)]
    
    print("> PCA: %i => %i dimension(s) with eigenval.: %s"%(len(headers), n_pca, str(eigenvalues)))
    print("sum of Variance: %s"%(np.sum(eigenvalues)))
    with open("%sPCA_coords.txt"%output_folder, "w") as f:
        for x in range(n_pca):
            f.write("PC%i(%.2f) "%(x + 1, eigenvalues[x]))
        f.write("\n")
        for el in reddim_coords:
            for n in el:
                f.write("%f "%n)
            f.write("\n")  
    return pca_obj, reddim_coords, original_coords


def calc_magn_vector(pc):
    final_length = int(len(pc)/3)
    pc_magn = np.empty(final_length)
    
    for i in range(final_length):
        pc_magn[i] = np.sqrt(pc[i*3]**2+pc[i*3+1]**2+pc[i*3+2]**2)
    
    return pc_magn


def save_pc_bfact(init_padding, length_pc, final_padding, structure, pc_component, outpdb):

    pc_elongated = np.concatenate((np.zeros(init_padding),calc_magn_vector(pc_component),np.zeros(final_padding)))
    system_length_res = len([residue for residue in structure.topology.residues])
    system_length_atoms = len([ atom for atom in structure.topology.atoms])

    syst_bfact=np.zeros(system_length_atoms)
    index=0
    for resid in range(system_length_res):
        res_len = len(structure.topology.select('resid '+str(resid)))
        res_bfact= res_len
        syst_bfact[index:index+res_len] = pc_elongated[resid]
        index += res_len

    structure[0].save_pdb(outpdb, bfactors=syst_bfact*100)
    
# Check convercence PC
def calculate_pca_convergence(topo, traj, main_folder, syst, at_sel, n_pca, nframes, nsim ):
    # Load trajectory
    topology = mdtraj.load(topo).topology
    struct_ens = mdtraj.load(traj, top=topo)

    # Create result folder based on dataset nam\e
    if not os.path.exists(main_folder):
        os.makedirs(main_folder)
    out_idx =  1
    output_folder = main_folder+"/%s_%i/"
    while os.path.exists(output_folder%(syst, out_idx)):
        out_idx += 1
    output_folder = output_folder%(syst, out_idx)
    os.makedirs(output_folder)
    
    selection = struct_ens.topology.select(at_sel)
    coords = struct_ens.xyz[:, selection]
    coords = coords.reshape(coords.shape[0], coords.shape[1] * coords.shape[2])
    headers = ["C%i"%x for x in range(len(coords[0]))]  # general headers
    
    frameslist = np.multiply(np.arange(1,nsim+1),nframes)
    original_coords = coords.copy()
    pcas_list = []
    reddim_coords_list = []
    print(np.shape(coords))
    # Principal component analysis
    for section in frameslist:
        pca_obj = PCA(n_components=n_pca)
        
        reddim_coords = pca_obj.fit_transform(coords[0:section])
        pcas_list.append(pca_obj)
        #eigenvalues = pca_obj.explained_variance_ratio_
        #ratio = np.sum(eigenvalues[:n_pca])
        #pca_headers = ["PC%i (%.2f)"%(x + 1, eigenvalues[x]) for x in range(n_pca)]
    
        #print("> PCA: %i => %i dimension(s) with eigenval.: %s"%(len(headers), n_pca, str(eigenvalues)))
        #print("sum of Variance: %s"%(np.sum(eigenvalues)))
  
    return pcas_list

ImportError: C extension: None not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

In [3]:
# PARAMETERS SETUP

# Clustering parameters 
pdc=3.5   # neighbour search during clusterint
n_resize=4 
filt=0.1 
verbose=False
n_pca=10 # number of PC to include in calculations 
n_bins= 200  # number of bins for PC hist
# Simulations and topology 
trj_apo='apo/trjcat_md_astera_apo_center_pbcmol_fit_rot_trans_CA_after10ns_stride10.xtc'   # apo traj
trj_holo='holo/holo_prot.xtc'  # holo traj
trj_tot='astera_tot.xtc'   # concatenated total traj

topo_apo='apo.gro'  # pdb or gro of apo

# Selection for PCA calculation (see mdtraj syntax)
at_sel="name CA and residue 374 to 534"  # try to exclude end loops or regions that move but are not interesting (hides the true PC)
n_res=160   # number of residues (still manual insertion, i know...)
resmin=374
resmax=534
feat="None"

output_folder = "results_div"

apo_name="apo"
holo_name="holo"
tot_name="tot"

In [4]:
# make results folder 
try:
    os.mkdir(output_folder)
except FileExistsError:
    pass
# make images folder 
try:
    os.mkdir(output_folder+'/IMAGES/')
except FileExistsError:
    pass
# Calculate PCA
pca_apo, red_dim_coords_apo, orig_coords_apo = calculate_pca(topo_apo, trj_apo, output_folder, apo_name, at_sel, n_pca)
pca_holo, red_dim_coords_holo, orig_coords_holo = calculate_pca(topo_apo, trj_holo, output_folder, holo_name, at_sel, n_pca)
pca_tot, red_dim_coords_tot, orig_coords_tot = calculate_pca(topo_apo, trj_tot, output_folder, tot_name, at_sel, n_pca)


> PCA: 483 => 10 dimension(s) with eigenval.: [0.1792123  0.11973875 0.08370548 0.06231228 0.0473187  0.04409574
 0.03407646 0.02994204 0.0261161  0.0249496 ]
sum of Variance: 0.6514675
> PCA: 483 => 10 dimension(s) with eigenval.: [0.21427467 0.13883297 0.08015925 0.05711839 0.05020268 0.04093185
 0.03756292 0.02969201 0.02788982 0.02247912]
sum of Variance: 0.69914365
> PCA: 483 => 10 dimension(s) with eigenval.: [0.17437395 0.14128675 0.06386527 0.05748935 0.05466145 0.04443454
 0.03584496 0.03262374 0.02790473 0.02248705]
sum of Variance: 0.65497184


In [6]:
index_apo_end=np.shape(red_dim_coords_apo)[0]
index_tot_end=np.shape(red_dim_coords_tot)[0]

In [7]:
import numpy as np
from scipy.stats import entropy

# Define two probability distributions (can be arrays, lists, or pandas Series)
p = np.array(red_dim_coords_tot[0:index_apo_end,0])
q = np.array(red_dim_coords_tot[index_apo_end:index_tot_end,0])

# Compute histograms to estimate the probability distributions
hist1, bin_edges1 = np.histogram(p, bins=200, density=True)
hist2, bin_edges2 = np.histogram(q, bins=200, density=True)

# Add a small constant to avoid zero values
epsilon = 1e-10
hist1 = hist1 + epsilon
hist2 = hist2 + epsilon

# Normalize distributions to ensure they sum to 1
hist1 /= np.sum(hist1)
hist2 /= np.sum(hist2)


# Compute KL divergence
kl_divergence1 = entropy(hist1, hist2)
kl_divergence2 = entropy(hist2, hist1)

kl_divergence = (kl_divergence1 + kl_divergence2) / 2

print(f"KL Divergence: {kl_divergence}")

KL Divergence: 0.13660804097619783
